In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os

In [2]:
#Base Url for New York, NY Data Science Jobs
URL = "https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY"
Base_URL = "https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY"
pages = '&start='
counter = 325


In [3]:
#Create the Python data frame
JobsB_df = pd.DataFrame(columns=['Title', 'Company', 'Description', 'Link'])


In [4]:
# 
# driver = webdriver.Firefox()
# print(URL)
# driver.get(URL)
    
# elem = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "jobtitle", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "turnstileLink", " " ))]')
# driver.implicitly_wait(30)
# for link in elem:
#     link.click()
#     try:
#         post = driver.find_element_by_xpath('//*[@id="vjs-desc"]').text
#         title = driver.find_element_by_xpath('//*[@id="vjs-jobtitle"]').text 
#         company = driver.find_element_by_xpath('//*[@id="vjs-cn"]/a').text
        
#     except:
#         pass
#     print(post)
#     print("\n")
# print(len(elem))
# print(elem)
# 

In [5]:
for i in range(1,1000):
    driver = webdriver.Firefox()
    print(URL)
    driver.get(URL)
    driver.implicitly_wait(20)
    elements = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "jobtitle", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "turnstileLink", " " ))]')
    
    for job in elements:
        try:
            job.click()
            post = driver.find_element_by_xpath('//*[@id="vjs-desc"]').text
            title = driver.find_element_by_xpath('//*[@id="vjs-jobtitle"]').text 
            company = driver.find_element_by_xpath('//*[@id="vjs-cn"]/a').text             
            JobsB_df = JobsB_df.append({'Title':title,'Company':company, 'Description':post, 'Link':URL}, ignore_index=True)
        except:
            pass
        
    driver.quit()
    counter = counter + i
    pages = '&start=' + str(counter)
    URL = Base_URL + pages   
    counter = 325
    
    

https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=326
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=327
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=328
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=329
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=330
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=331
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=332
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=333
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=334
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=335
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=336
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=337
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=338
https://www.indeed.com/jobs?q=

https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=443
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=444
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=445
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=446
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=447
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=448
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=449
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=450
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=451
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=452
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=453
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=454
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=455
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=456
https://www.indeed.c

https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=561
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=562
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=563
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=564
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=565
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=566
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=567
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=568
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=569
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=570
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=571
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=572
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=573
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=574
https://www.indeed.c

https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=679
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=680
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=681
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=682
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=683
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=684
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=685
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=686
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=687
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=688
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=689
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=690
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=691
https://www.indeed.com/jobs?q=Data+Science&l=New+York%2C+NY&start=692
https://www.indeed.c

WebDriverException: Message: Failed to decode response from marionette


In [6]:
JobsB_df

,Title,Company,Description,Link
0,Data Science Quant Developer | 2019 Grad,Liquidnet,With the acquisition of OTAS Technologies in 2...,https://www.indeed.com/jobs?q=Data+Science&l=N...
1,DATA SCIENCE -INTERN,"Foot Locker, Inc.","OVERVIEW\nFoot Locker, Inc. is the world’s lea...",https://www.indeed.com/jobs?q=Data+Science&l=N...
2,"Data Science Internship, Summer 2019",Etsy,Etsy is looking for Data Science Interns to jo...,https://www.indeed.com/jobs?q=Data+Science&l=N...
3,"Internships, Data Strategy/Data Science- Summe...",Viacom,Overview\nPlease note: these departments prima...,https://www.indeed.com/jobs?q=Data+Science&l=N...
4,ASSOCIATE DATA SCIENTIST,"Foot Locker, Inc.",OVERVIEW\nThe Associate Data Scientist assists...,https://www.indeed.com/jobs?q=Data+Science&l=N...
5,Data Science Analyst - Genetics and Genomic Sc...,Mount Sinai Health System,Strength Through Diversity\nGround breaking sc...,https://www.indeed.com/jobs?q=Data+Science&l=N...
6,Music Research Analyst,Nielsen,Music Research Analyst - 28092\nAnalytics - US...,https://www.indeed.com/jobs?q=Data+Science&l=N...
7,"Data Engineer, ETL",Marketing Evolution,Company Overview:\n\n\nMarketing Evolution has...,https://www.indeed.com/jobs?q=Data+Science&l=N...
8,Data Engineer,Comcast,Comcast brings together the best in media and ...,https://www.indeed.com/jobs?q=Data+Science&l=N...
9,Full-time Research Engineer at the NYU Visuali...,New York University,New York University: NYU - Domestic: Tandon Sc...,https://www.indeed.com/jobs?q=Data+Science&l=N...


In [7]:
Final_Jobs_df=JobsB_df.drop_duplicates(['Company'], keep='first')

In [8]:
Final_Jobs_df.to_csv("Indeed Data Science Jobs Revised3.csv")

In [10]:
Final_Jobs_df_01 = Final_Jobs_df

In [11]:
Final_Jobs_df_02 = pd.read_csv("Indeed Data Science Jobs Revised2.csv")

In [13]:
Final_Jobs_df_02

,Unnamed: 0,Title,Company,Description,Link
0,0,Data Science Quant Developer | 2019 Grad,Liquidnet,With the acquisition of OTAS Technologies in 2...,https://www.indeed.com/jobs?q=Data+Science&l=N...
1,1,"Internships, Data Strategy/Data Science- Summe...",Viacom,Overview\r\nPlease note: these departments pri...,https://www.indeed.com/jobs?q=Data+Science&l=N...
2,2,"Data Science Internship, Summer 2019",Etsy,Etsy is looking for Data Science Interns to jo...,https://www.indeed.com/jobs?q=Data+Science&l=N...
3,3,Data Science Undergraduate Internship,Aetna,What it is\r\nAetna’s internship experience is...,https://www.indeed.com/jobs?q=Data+Science&l=N...
4,4,Data Science Analyst - Genetics and Genomic Sc...,Mount Sinai Health System,Strength Through Diversity\r\nGround breaking ...,https://www.indeed.com/jobs?q=Data+Science&l=N...
5,5,"VP, Data Science",Plated,Come to Plated and make data science.\r\n\r\nY...,https://www.indeed.com/jobs?q=Data+Science&l=N...
6,6,Data Engineer,Amazon.com,Job Description\r\nLove food? We do! The Amazo...,https://www.indeed.com/jobs?q=Data+Science&l=N...
7,7,"Intern, Data Science - Engineering - Part-Time",SiriusXM,Position Type: Internship\r\nSession: Spring 2...,https://www.indeed.com/jobs?q=Data+Science&l=N...
8,8,Junior Data Scientist,Dow Jones,Job Description:\r\nWe are looking for a junio...,https://www.indeed.com/jobs?q=Data+Science&l=N...
9,9,Data Scientist,Roku,About Roku:\r\nRoku pioneered streaming to the...,https://www.indeed.com/jobs?q=Data+Science&l=N...


In [15]:
Data_Science_Indeed_Jobs = Final_Jobs_df_01.append(Final_Jobs_df_02)

In [17]:
Data_Science_Indeed_Jobs = Data_Science_Indeed_Jobs[["Title", "Company", "Description","Link"]]


In [29]:
Data_Science_Indeed_Jobs=Data_Science_Indeed_Jobs.reset_index(drop=True)

In [30]:
Data_Science_Indeed_Jobs.to_csv("Data_Science_Indeed_Jobs.csv")